In [1]:
import pandas as pd
import glob
import swifter
path = r'/data/semeval-2020/task-11/datasets/train-tagged_article/'
all_files = glob.glob(path + "/*.txt")

content_tuple = []

for filename in all_files:
    file = open(filename)
    content = "".join(file.readlines()).replace("\n"," ")
    article_id = filename.split("article")[-1].split(".txt")[0]
    content_tuple.append((article_id,content))

In [2]:
content_df = pd.DataFrame(content_tuple,columns=["article_id","content"])

In [3]:
content_df.head(3)

,article_id,content
0,762956953,Iran Admits To Aiding Al-Qaeda and Facilitatin...
1,787529309,The Last-Minute <span-11 Character Assassinati...
2,999001296,Altered Election Documents Tied To Florida Dem...


In [4]:
import spacy; from spacy.lang.en import English; nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))

def create_sentences(row):
    dataset = []
    article_id = row.article_id
    doc = nlp(row.content)
    sentences = list(doc.sents)
    for sent in sentences:
        dataset.append((article_id,str(sent)))
    return dataset

In [5]:
a = content_df.swifter.apply(lambda x: create_sentences(x),axis=1).values.tolist()

/home/raghavan/anaconda3/envs/semeval_2020/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
a[:3]

[[('762956953',
   'Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks  This has long been known, although the mainstream media dismissed it as a conspiracy theory.'),
  ('762956953', 'But now we have definitive confirmation.'),
  ('762956953', 'It was Iran Bush should have invaded after 9/11 , not Iraq.'),
  ('762956953',
   'Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic.'),
  ('762956953',
   'The Iran nuclear deal should never have proceeded — President Obama, <span-6 the worst president in American history 6-/span> . “'),
  ('762956953',
   'Iran Admits To Facilitating 9/11 Terror Attacks,” by Adam Kredo, Washington Free Beacon, June 8, 2018: Iranian officials, in a first, have admitted to facilitating the 9/11 terrori

In [7]:
article_id_sentences = pd.DataFrame([c for b in a for c in b],columns=["article_id","sentence"])

In [8]:
article_id_sentences["sentence_span_count"] = article_id_sentences.sentence.swifter.apply(lambda x: x.count("span"))

In [9]:
article_id_sentences["has_odd_span"] = article_id_sentences.sentence_span_count.swifter.apply(lambda x: x%2)

In [10]:
len(set(article_id_sentences[article_id_sentences["has_odd_span"] == 1].article_id.values.tolist()))

222

In [11]:
article_id_sentences[article_id_sentences["has_odd_span"] == 1].sentence.values.tolist()[:10]

['<span-0 The information was sent on Nov. 9 by Bradley McVay, DOS’ interim general counsel, who asked that the altered dates be investigated. “',
 'Altering a form in a manner that provides the incorrect date for a voter to cure a defect … imposes a burden on the voter significant enough to frustrate the voter’s ability to vote,” McVay wrote in a letter that was sent Nov. 9 and released publicly on Tuesday 0-/span>.',
 '<span-7 He also sang an Islamic State fight song and recalled making a replica of the group’s flag.',
 'He said he wanted to replace his neighbor’s American flag with it 7-/span>.',
 '<span-5 In the process, however, he misidentified Hasan as Nidal Hussein and said the shooting happened at Fort Knox in Kentucky.',
 'He also said Hasan had died <span-9 a martyr9-/span>, though in fact he remains alive on military death row 5-/span>.',
 'According to Slate, the image quickly made the rounds on the internet; it currently has almost 9,000 retweets and many speculate that t

In [12]:
article_id_sentences["propaganda"] = article_id_sentences.sentence_span_count.swifter.apply(lambda x: x > 0)

In [13]:
article_id_sentences.head(3)

,article_id,sentence,sentence_span_count,has_odd_span,propaganda
0,762956953,Iran Admits To Aiding Al-Qaeda and Facilitatin...,0,0,False
1,762956953,But now we have definitive confirmation.,0,0,False
2,762956953,It was Iran Bush should have invaded after 9/1...,0,0,False


In [14]:
import re
def get_classes(article):
    results_start_span = re.findall("<span-\d+",article)
    results_end_span = re.findall("\d+-/span>",article)
    start_span_classes = [int(i.replace("<span-","")) for i in results_start_span]
    end_span_classes = [int(i.replace("-/span>","")) for i in results_end_span]
    classes = list(set(start_span_classes + end_span_classes))
    return classes

def clean_sentences(article):
    return re.sub("<span-\d+","",re.sub("\d+-/span>","",article))

In [15]:
article_id_sentences["classes"] = article_id_sentences.sentence.swifter.apply(get_classes)
article_id_sentences["cleaned_sentence"] = article_id_sentences.sentence.swifter.apply(clean_sentences)


In [16]:
article_id_sentences.head(434).tail(2)

,article_id,sentence,sentence_span_count,has_odd_span,propaganda,classes,cleaned_sentence
432,732154721,Hungarian Prime Minister: ‘<span-7 <span-11 Ch...,6,0,True,"[11, 7]",Hungarian Prime Minister: ‘ Christianity is E...
433,732154721,”11-/span> 7-/span> Addressing his country and...,3,1,True,"[11, 7]","” Addressing his country and the world, Prime..."


In [17]:
article_id_sentences[["cleaned_sentence","propaganda","classes"]].to_csv("/data/semeval-2020/task-11/processed/sentence_dataset.csv",index=False)

In [21]:
sample_sentence = '”11-/span> 7-/span> Addressing his country and the world, Prime Minister Viktor Orbán warned that <span-7 European nations which have encouraged migration have, “opened the way to the decline of Christian culture and the advance of Islam.”'

In [25]:
re.sub("\d+-/span>","",sample_sentence)

'”  Addressing his country and the world, Prime Minister Viktor Orbán warned that <span-7 European nations which have encouraged migration have, “opened the way to the decline of Christian culture and the advance of Islam.”'